# How to complete this assignment
First, download [this Kaggle dataset](https://www.kaggle.com/hugomathien/soccer) and extract *sqlite* database. You may need to register at https://www.kaggle.com/ beforehand. Then complete 15 graded tasks below, the score is given in brackets. Finally submit the resulting `.ipynb` file to rs-app Auto-test.

- Do not delete or rename the variables given before the inscription `#your code here`, they are needed for the correct verification.
- Do not change the code in the last Notebook cell, it is required for the server check.
- Your Notebook must run completely without errors to be graded! Please check everything before submission by going *Cell -> Run All*

## Some important notes
- If you need to **calculate the number of "something"** that means we expect you to assign an Integer to the given variable
- If you need to **make a list of "something"** we expect you to assign a Python list with appropriate values to the given variable
- If you need to find a **specifiс player, day of the week, team, etc.** we expect you to assign a String with the full name of the entity to the given variable (`player_name`, day of week full name, `team_name`, etc.)

In [1]:
import sqlite3
import pandas as pd
import os

pd.set_option('display.max_column', None)

In [2]:
# Leave that code unchanged, it is required for the server check!
db = sqlite3.connect(os.environ.get("DB_PATH") or 'database.sqlite')

In [3]:
# You may load the data from SQL table directly to the Pandas dataframe as
player_data = pd.read_sql("SELECT * FROM Player;", db)
player_stats_data = pd.read_sql("SELECT * FROM Player_Attributes;", db)
team_data = pd.read_sql("SELECT * FROM Team;", db)
match_data = pd.read_sql("SELECT * FROM Match;", db)
league_data = pd.read_sql("SELECT * FROM LEAGUE", db)

In [4]:
player_data.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


**Task 1 (0.25 point).** Calculate the number of players with a height between 180 and 190 inclusive

In [5]:
players_180_190 = player_data[(player_data['height'] >= 180)&(player_data['height'] <= 190)].shape[0]
players_180_190# Your code here

5925

In [6]:
assert(isinstance(players_180_190, int))

**Task 2 (0.25 point).** Calculate the number of players born in 1980. <br>
**Hint:** you may want to cast your 'birthday' column to DateTime type by [pandas.to_datetime](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)

In [7]:
player_data['birthday'] = pd.to_datetime(player_data['birthday'])
players_1980 = player_data[player_data['birthday'].dt.year == 1980].shape[0]
players_1980# Your code here

434

In [8]:
assert(isinstance(players_1980, int))

**Task 3 (0.25 point).** Make a list of the top 10 players with the highest weight sorted in descending order. If there are several players with the same weight put them in the lexicographic order by name.

In [9]:
highest_players = player_data.sort_values(by=['weight', 'player_name'], ascending = [False, True])[:10]['player_name'].tolist()
highest_players# Your code here

['Kristof van Hout',
 'Tim Wiese',
 'Jeroen Verhoeven',
 'Ishmael Miller',
 'Cassio',
 'Christopher Samba',
 'Lars Unnerstall',
 'Marcus Hahnemann',
 'Abdoulaye Faye',
 'Enoch Showunmi']

In [10]:
assert(len(highest_players) == 10)
assert(isinstance(highest_players, list))
for i in range(10):
    assert(isinstance(highest_players[i], str))

**Task 4 (0.5 point).** Make a list of tuples containing years along with the number of players born in that year from 1980 up to 1990. <br>
**Structure example**: [(1980, 123), (1981, 140) ..., (1990, 83)] -> There were born 123 players in 1980, there were born 140 players in 1981 and etc.

In [11]:
player_data['birthday_year'] = player_data['birthday'].dt.year
born_years = player_data.groupby('birthday_year').size().loc[lambda x: ( 1980 <= x.index)&(x.index <= 1990)]
years_born_players = list(born_years.items())
years_born_players# Your code here

[(1980, 434),
 (1981, 475),
 (1982, 487),
 (1983, 541),
 (1984, 607),
 (1985, 593),
 (1986, 710),
 (1987, 722),
 (1988, 802),
 (1989, 723),
 (1990, 696)]

In [12]:
assert(len(years_born_players) == 11)
assert(isinstance(years_born_players, list))
for i in range(10):
    assert(isinstance(years_born_players[i], tuple))
    assert(isinstance(years_born_players[i][0], int))
    assert(isinstance(years_born_players[i][1], int))

**Task 5 (0.5 point).** Calculate the mean and the standard deviation of the players' **height** with the name **Adriano**. <br>
**Note:** Name is represented by the first part of `player_name`.

In [13]:
mask = player_data['player_name'].str.startswith('Adriano')
adriano_mean, adriano_std = player_data[mask]['height'].mean(), player_data[mask]['height'].std()
(adriano_mean, adriano_std)# Your code here

(182.11800000000002, 5.361479480723788)

In [14]:
assert(isinstance(adriano_mean, float))
assert(isinstance(adriano_std, float))

**Task 6 (0.75 point).** How many players were born on each day of the week? Find the day of the week with the minimum number of players born.

In [15]:
dow_with_min_players_born = player_data.groupby(player_data['birthday'].dt.day_name()).size().sort_values().keys()[0]
dow_with_min_players_born# Your code here

'Sunday'

In [16]:
assert(isinstance(dow_with_min_players_born, str))

**Task 7 (0.75 point).** Find a league with the most matches in total. If there are several leagues with the same amount of matches, take the first in the lexical order.

In [17]:
league = league_data[['id', 'name']]
league.columns = ['league_id', 'league_name']

league_most_matches = match_data \
    .merge(league, 'left') \
    .groupby('league_name').size() \
    .reset_index() \
    .sort_values(by=[0, 'league_name'], ascending = [False, True]).iloc[0]['league_name']
league_most_matches # Your code here

'England Premier League'

In [18]:
assert(isinstance(league_most_matches, str))

**Task 8 (1.25 point).** Find a player who participated in the largest number of matches during the whole match history. Assign a `player_name` to the given variable

In [19]:
import re
home_player_cols = [c for c in match_data.columns if re.match(r'home_player_\d\d?', c)]
away_player_cols = [c for c in match_data.columns if re.match(r'away_player_\d\d?', c)]
player_cols = home_player_cols + away_player_cols

max_matches_player_id = match_data.melt(value_vars=player_cols).groupby('value').size().sort_values(ascending=False).keys()[0]

max_matches_player = player_data[player_data['player_api_id'] == max_matches_player_id]['player_name'].values[0]
max_matches_player# Your code here

'Steve Mandanda'

In [20]:
assert(isinstance(max_matches_player, str))

**Task 9 (1.5 point).** List top-5 tuples of most correlated **player's characteristics** in the descending order of the absolute [Pearson's coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) value.

**Note 1:** Players characteristics are all the columns in `Player_Attributes` table except `[id, player_fifa_api_id, player_api_id, date, preferred_foot, attacking_work_rate, defensive_work_rate]`). <br>
**Note 2:** Exclude duplicated pairs from the list. E.g. ('gk_handling', 'gk_reflexes') and ('gk_reflexes', 'gk_handling') are duplicates, leave just one of them in the resulting list.

**Hint:** You may use [dataframe.corr()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) for calculating pairwise Pearson correlation.

In [26]:
other_cols = ['id', 'player_fifa_api_adi', 'player_api_adi', 'date', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate']
char_cols = [c for c in player_stats_data if c not in other_cols]
corr = player_stats_data[char_cols].corr().abs().unstack().reset_index()
corr = corr[corr['level_0'] != corr['level_1']]
corr['feature_pair'] = corr.apply(lambda x: (x['level_0'], x['level_1']), axis = 1)
corr['feature_pair'] = corr.apply(lambda x: tuple(sorted(x['feature_pair'])), axis=1)
top_correlated_features = corr[['feature_pair', 0]].drop_duplicates().sort_values(by=0, ascending=False)[:5]['feature_pair'].values.tolist()
top_correlated_features# Your code here

[('gk_positioning', 'gk_reflexes'),
 ('gk_handling', 'gk_reflexes'),
 ('gk_handling', 'gk_positioning'),
 ('sliding_tackle', 'standing_tackle'),
 ('marking', 'standing_tackle')]

In [ ]:
assert(len(top_correlated_features) == 5)
assert(isinstance(top_correlated_features, list))
for i in range(5):
    assert(isinstance(top_correlated_features[i], tuple))
    assert(isinstance(top_correlated_features[i][0], str))
    assert(isinstance(top_correlated_features[i][1], str))

**Task 10 (2 points).** Find top-5 most similar players to **Neymar** whose names are given. The similarity is measured as [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) between vectors of players' characteristics (described in the task above). Put their names in a vector in ascending order by Euclidean distance and sorted by `player_name` if the distance is the same<br>
**Note 1:** There are many records for some players in the `Player_Attributes` table. You need to take the freshest data (characteristics with the most recent `date`). <br>
**Note 2:** Use pure values of the characteristics even if you are aware of such preprocessing technics as normalization. <br>
**Note 3:** Please avoid using any built-in methods for calculating the Euclidean distance between vectors, think about implementing your own.

In [39]:
import numpy as np
players_extended = player_stats_data.merge(player_data[['player_api_id', 'player_name']], 'inner', 'player_api_id') \
    .sort_values('date').drop_duplicates(['player_api_id'], keep='last')

name = 'Neymar'
neymar_vector = players_extended[players_extended['player_name'] == name][char_cols].values[0]
player_vector = players_extended[char_cols].values
players_extended['neymar_similar'] = np.sqrt(np.sum((player_vector - neymar_vector)**2, axis = 1))

neymar_similarities = players_extended \
    .sort_values(by=['neymar_similar', 'player_name'], ascending = True)['player_name'][1:6].values.tolist()
neymar_similarities# Your code here

['Rivaldo', 'Michihirio Yasuda', 'Joao Guilherme', 'Hulk', 'Edivaldo Rojas']

In [41]:
assert(len(neymar_similarities) == 5)
assert(isinstance(neymar_similarities, list))
for i in range(5):
    assert(isinstance(neymar_similarities[i], str))

**Task 11 (1 point).** Calculate the number of home matches played by the **Borussia Dortmund** team in **Germany 1. Bundesliga** in season **2008/2009**

In [42]:
league = league_data[['id', 'name']]
league.columns = ['league_id', 'league_name']
match_league = match_data.merge(league, on=['league_id'])

home_teams = team_data[['team_api_id', 'team_long_name']].rename(columns={'team_api_id': 'home_team_api_id', 'team_long_name': 'home_team_name'})
away_teams = team_data[['team_api_id', 'team_long_name']].rename(columns={'team_api_id': 'away_team_api_id', 'team_long_name': 'away_team_name'})
match_team_league = match_league.merge(home_teams, 'left', ['home_team_api_id']).merge(away_teams, 'left', ['away_team_api_id'])

borussia_bundesliga_2008_2009_matches = match_team_league[(match_team_league['league_name'] == 'Germany 1. Bundesliga')&(match_team_league['season'] == '2008/2009')&(match_team_league['home_team_name'] == 'Borussia Dortmund')].shape[0]
borussia_bundesliga_2008_2009_matches# Your code here

17

In [43]:
assert(isinstance(borussia_bundesliga_2008_2009_matches, int))

**Task 12 (1 point).** Find a team having the most matches (both home and away!) in the **Germany 1. Bundesliga** in **2008/2009** season. Return number of matches.

In [46]:
match_team_target_league_season = match_team_league[(match_team_league['league_name'] == 'Germany 1. Bundesliga') & (match_team_league['season'] == '2008/2009')]

team_most_matches_bundesliga_2008_2009 = match_team_target_league_season.groupby('home_team_name').size().add(match_team_target_league_season.groupby('away_team_name').size()).sort_values(ascending = False)[0].item()
team_most_matches_bundesliga_2008_2009# Your code here

34

In [47]:
assert(isinstance(team_most_matches_bundesliga_2008_2009, int))

**Task 13 (1 point).** Count total number of **Arsenal** matches (both home and away!) in the **2015/2016** season which they have won. <br><br>
**Note:** Winning a game means scoring **more** goals than an opponent.

In [50]:
def team_won(row):
    if row['home_team_goal']>row['away_team_goal']:
        return row['home_team_name']
    elif row['home_team_goal']<row['away_team_goal']:
        return row['away_team_name']
    else:
        return None
match_team_league['team_won'] = match_team_league.apply(lambda x:team_won(x), axis=1)

arsenal_won_matches_2015_2016 = match_team_league[(match_team_league['team_won'] == 'Arsenal') & (match_team_league['season'] == '2015/2016')].shape[0]
arsenal_won_matches_2015_2016# Your code here

20

In [ ]:
assert(isinstance(arsenal_won_matches_2015_2016, int))

**Task 14 (2 points).** Find a team with the highest win rate in the **2015/2016** season. Win rate means won matches / all matches. If there are several teams with the highest win rate return the first by name in lexical order

In [55]:
target_matches = match_team_league[match_team_league['season'] =='2015/2016']
team_total_plays = target_matches.groupby('home_team_name').size().add(target_matches.groupby('away_team_name').size()).to_frame()

team_total_plays.columns = ['total_matches']
team_total_plays.index.names = ['team_name']

team_won_playes = target_matches.groupby('team_won').size().to_frame()
team_won_playes.columns = ['won_matches']

matches_won_total = team_total_plays.join(team_won_playes)
matches_won_total['win_rate'] = matches_won_total['won_matches'] / matches_won_total['total_matches']

team_highest_winrate_2015_2016 = matches_won_total.sort_values(['win_rate', 'team_name'], ascending = [False, True])[:1].index[0]
team_highest_winrate_2015_2016# Your code here

'SL Benfica'

In [ ]:
assert(isinstance(team_highest_winrate_2015_2016, str))

**Task 15 (2 points).** Determine the team with the maximum days' gap between matches in **England Premier League 2010/2011 season**. Return number of days in that gap. <br>
**Note**: a *gap* means the number of days between two consecutive matches of the same team.

In [57]:
match_team_league['date'] = pd.to_datetime(match_team_league['date'])



highest_gap_england_2010_2011

Timedelta('546 days 00:00:00')

In [ ]:
assert(isinstance(highest_gap_england_2010_2011, int))

### Warning! Do not change anything in the area below

In [ ]:
with open('student_answers.txt', 'w') as file:
    file.write(f"{players_180_190}\n")
    file.write(f"{players_1980}\n")
    file.write(f"{highest_players}\n")
    file.write(f"{years_born_players}\n")
    file.write(f"{round(adriano_mean, 3)} {round(adriano_std, 3)}\n")
    file.write(f"{dow_with_min_players_born}\n")
    file.write(f"{league_most_matches}\n")
    file.write(f"{max_matches_player}\n")
    file.write(f"{';'.join(['%s,%s' % tup for tup in top_correlated_features])};\n")
    file.write(f"{neymar_similarities}\n")
    file.write(f"{borussia_bundesliga_2008_2009_matches}\n")
    file.write(f"{team_most_matches_bundesliga_2008_2009}\n")
    file.write(f"{arsenal_won_matches_2015_2016}\n")
    file.write(f"{team_highest_winrate_2015_2016}\n")
    file.write(f"{highest_gap_england_2010_2011}\n")